<a href="https://colab.research.google.com/github/wuchaojie925/no/blob/master/read_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.


In [2]:
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
from tensorflow import feature_column
from tensorflow.keras import layers

In [0]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')

In [0]:
 def get_set(train_df):
    key_set = {}
    key_set['string'] = {}
    key_set['int'] = {}
    key_set['float'] = {}

    for key in train_df.keys():
        if train_df[key].dtypes == 'object':
            key_set['string'][key] = []
            train_df[key] = train_df[key].fillna('UNK')
            key_set['string'][key].append(list(set(train_df[key].values.tolist())))
        elif train_df[key].dtypes == 'int64':
            key_set['int'][key] = []
            train_df[key] = train_df[key].fillna(0)
            key_set['int'][key].append(list(set(train_df[key].values.tolist())))
        elif train_df[key].dtypes == 'float64':
            key_set['float'][key] = []
            train_df[key] = train_df[key].fillna(0.0)
            key_set['float'][key].append(list(set(train_df[key].values.tolist())))
    return key_set

In [0]:
key_set = get_set(train_df)
key_set['int'].pop('Survived')

In [0]:
for type in key_set:
    for key in key_set[type]:
        key_set[type][key] = key_set[type][key][0]

In [0]:
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [0]:
def get_ds(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    label = dataframe.pop('Survived')
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), label))
    if shuffle:
        dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.batch(batch_size)
    return dataset

In [0]:
train_ds = get_ds(train_df, shuffle=False, batch_size=32)
val_ds = get_ds(val_df, shuffle=False, batch_size=32)


In [0]:
feature_columns = []

for key in key_set['int']:
    feature_columns.append(feature_column.numeric_column(key))

for key in key_set['float']:
    feature_columns.append(feature_column.numeric_column(key))

for key in key_set['string']:
    keys = feature_column.categorical_column_with_vocabulary_list(key, key_set['string'][key])
    embedding = feature_column.embedding_column(keys, dimension=8)
    feature_columns.append(embedding)


In [86]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(32, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=22)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/22
26/26 [==============================] - 3s 122ms/step - loss: 0.9766 - accuracy: 0.6105 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/22
26/26 [==============================] - 0s 5ms/step - loss: 0.7016 - accuracy: 0.6454 - val_loss: 0.9348 - val_accuracy: 0.6000
Epoch 3/22
26/26 [==============================] - 0s 5ms/step - loss: 0.6629 - accuracy: 0.6629 - val_loss: 0.8740 - val_accuracy: 0.6000
Epoch 4/22
26/26 [==============================] - 0s 5ms/step - loss: 0.6366

In [81]:

tf.executing_eagerly()

True